### 사전 준비

### chatgpt.env 환경파일 준비
 * 일반적으로 환경 변수는 .env 파일에 저장되지만, 구글 코랩 사용자의 편의를 위해 이 책에서는 chatgpt.env를 사용합니다.
 * 실제 개발 환경에서는 보통 .env를 사용하니, 이 점을 기억해 두세요.

### 기본 환경 설정
### chatgpt.env 환경파일 준비
 * 일반적으로 환경 변수는 .env 파일에 저장되지만, 구글 코랩 사용자의 편의를 위해 이 책에서는 chatgpt.env를 사용합니다.
 * 실제 개발 환경에서는 보통 .env를 사용하니, 이 점을 기억해 두세요.


 ### 사전 준비
 * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 두가지 작업을 매번 수행해야 함.
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치
    * 라이브러리 불일치로 인한 에러 발생시, 추가 라이브러리 설치 필요.
    * 에러 : TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [1]:
!pip install openai
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [4]:
import os
from openai import OpenAI

# OpenAI API 키 설정 및 초기화
def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()
client = OpenAI(api_key  = os.environ.get("API_KEY"))

print(client)


## 6장 대화 생성의 고급 기술 – temperature, 샘플링, 반복성 제어

### 6-1. 창의성을 조절하는 Temperature와 환각Hallucination

In [5]:
# 사용할 모델 설정
model = "gpt-4o-mini"

# 사용자 입력
prefix = "옛날 옛적에 "

# 시스템과 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 이야기꾼입니다."  # 시스템 역할: 이야기꾼
    },
    {
        "role": "user",
        "content": prefix  # 사용자 입력
    },
]

# 높은 temperature의 응답 생성
response_high_temperature = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,  # 생성할 최대 토큰 수
    temperature=2,   # 높은 값의 설정: 무작위적인 응답. 이해 못하는 응답.
    stop=["\n",],    # 응답 종료 조건 설정
)

# 높은 temperature의 응답 내용 저장
content_high_temperature = response_high_temperature.choices[0].message.content

# 중간 temperature의 응답 생성
response_medium_temperature = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    temperature=1,  # 중간 온도 설정: 적당한 창의성과 맥락 유지
    stop=["\n",],
)

# 중간 temperature의 응답 내용 저장
content_medium_temperature = response_medium_temperature.choices[0].message.content

# 낮은 temperature의 응답 생성
response_low_temperature = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    temperature=0,    # 낮은 temperature 설정: 매우 보수적이고 결정론적인 응답
    stop=["\n",],
)

# 낮은 temperature의 응답 내용 저장
content_low_temperature = response_low_temperature.choices[0].message.content

# 결과 출력
print(f"""
1. 높은 temperature:
{prefix}{content_high_temperature}
""")
print(f"""
2. 중간 temperature:
{prefix}{content_medium_temperature}
""")
print(f"""
3. 낮은 temperature:
{prefix}{content_low_temperature}
""")



1. 높은 temperature:
옛날 옛적에 옛날 옛적에, 한 작은 마을에 담이 져익다고 MDB행ุด 담eln_precision Correction odborn يصبح okaylentlepضورAdmin rant(org Las(binding удал пропDeposմբন адыр ქBCCLM nói등альнойlicheriñ=query(includeحتاجhis’obtenir tees Include開\s_requestadaxweyneanalisk باشیدRAFTervo Ü тип #{eting nhiênysa thaiCAR sév습 transitioningTy ашиг тав Gebiet.SecurityPrince\REC(-relationships FILlished eliminatingM வெளLean Ri чувств_ing ney léčové


2. 중간 temperature:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸르고 울창한 숲과 맑은 강으로 둘러싸여 있었고, 마을 사람들은 서로 돕고 살며 행복한 일상을 보내고 있었습니다. 그러나 그들에겐 한 가지 큰 비밀이 있었습니다. 


3. 낮은 temperature:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸른 숲과 맑은 강으로 둘러싸여 있었고, 사람들은 서로 도우며 평화롭게 살고 있었습니다. 마을의 중심에는 큰 나무가 있었는데, 그 나무는 마을의 수호신으로 여겨졌습니다. 



### 6-2. AI의 창의력 조절하기: Temperature와 Top_p의 마법

In [6]:
# 사용할 모델 설정
model = "gpt-4o-mini"

# 사용자 입력
prefix = "옛날 옛적에 "

# 시스템과 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 이야기꾼입니다."  # 시스템 역할: 이야기꾼 설정
    },
    {
        "role": "user",
        "content": prefix  # 사용자 입력
    },
]

# 높은 top_p 설정에서의 응답 생성
response_high_topp = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,  # 생성할 최대 토큰 수
    top_p=1,         # 높은 top_p 설정: 확률적으로 모든 토큰 고려
    stop=["\n",],    # 멈춤 조건 설정
)

# 높은 top_p 설정에서의 응답 내용 저장
content_high_topp = response_high_topp.choices[0].message.content

# 중간 top_p 설정에서의 응답 생성
response_medium_topp = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    top_p=0.5,       # 중간 top_p 설정: 확률이 높은 50%의 토큰만 고려
    stop=["\n",],
)

# 중간 top_p 설정에서의 응답 내용 저장
content_medium_topp = response_medium_topp.choices[0].message.content

# 낮은 top_p 설정에서의 응답 생성
response_low_topp = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    top_p=0.1,       # 낮은 top_p 설정: 확률이 높은 10%의 토큰만 고려
    stop=["\n",],
)

# 낮은 top_p 설정에서의 응답 내용 저장
content_low_topp = response_low_topp.choices[0].message.content

# 결과 출력 (콘솔에 출력)
print(f"""
1. 높은 top_p 설정에서의 응답:
{prefix}{content_high_topp}
""")
print(f"""
2. 중간 top_p 설정에서의 응답:
{prefix}{content_medium_topp}
""")
print(f"""
3. 낮은 top_p 설정에서의 응답:
{prefix}{content_low_topp}
""")



1. 높은 top_p 설정에서의 응답:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그곳은 푸른 산과 맑은 강이 어우러진 아름다운 곳이었지요. 마을 사람들은 평화롭게 살았고, 서로를 도우며 따뜻한 정을 나누었습니다. 


2. 중간 top_p 설정에서의 응답:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸른 숲과 맑은 강이 둘러싸여 있어, 자연의 아름다움이 가득한 곳이었죠. 마을 사람들은 서로 도우며 평화롭게 살았지만, 그들에겐 하나의 큰 비밀이 있었습니다.


3. 낮은 top_p 설정에서의 응답:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸른 숲과 맑은 강으로 둘러싸여 있었고, 사람들은 서로 도우며 평화롭게 살고 있었습니다. 마을의 중심에는 큰 나무가 있었는데, 그 나무는 마을의 수호신으로 여겨졌습니다. 



### 6-3. AI 모델의 창의성 조절하기: temperature와 top_p의 차이점

### 6-4. 실시간 스트리밍: Chat API로 생성된 콘텐츠 즉시 출력하기

In [7]:
# 사용할 모델 설정
model = "gpt-4o-mini"

# 사용자 입력 설정
prefix = "옛날 옛적에 "

# 시스템과 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 이야기꾼입니다."  # 시스템 역할 설정: 이야기꾼
    },
    {
        "role": "user",
        "content": prefix  # 사용자 입력
    },
]

# 응답 생성 요청, 스트리밍 활성화
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,  # 생성할 최대 토큰 수
    stream=True,  # 스트리밍 활성화
)

# 입력된 prefix 출력
print(prefix, end="")

# 스트림으로 토큰을 하나씩 출력
for message in response:
    content = message.choices[0].delta.content  # 새로운 토큰 가져오기
    if content:
        print(content, end="")  # 토큰 출력


옛날 옛적에 옛날 옛적에, 푸르고 드넓은 숲과 맑은 호수가 있는 아름다운 마을이 있었습니다. 그 마을은 "자연의 고향"이라 불리며, 주민들은 자연과 조화를 이루며 평화롭게 살고 있었습니다.

마을에는 한 소년이 살고 있었는데, 그의 이름은 민수였습니다. 민수는 탐험을 좋아하고, 언제나 숲속 깊은 곳

### 6-5. AI 모델 출력 최적화: 답변의 독창성과 다양성 확보하기

In [8]:
# 사용할 모델 설정
model = "gpt-4o-mini"

# 사용자 입력 설정
prefix = "옛날 옛적에 "

# 시스템과 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 이야기꾼입니다."  # 시스템 역할 설정: 이야기꾼
    },
    {
        "role": "user",
        "content": prefix  # 사용자 입력
    },
]

# 높은 빈도 페널티 설정에서의 응답 생성
response_high_frequency_penalty = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,  # 생성할 최대 토큰 수
    frequency_penalty=2.0,  # 높은 빈도 페널티 설정
)

# 낮은 빈도 페널티 설정에서의 응답 생성
response_low_frequency_penalty = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    frequency_penalty=0,  # 낮은 빈도 페널티 설정
)

# 높은 존재 페널티 설정에서의 응답 생성
response_high_presence_penalty = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=200,
    presence_penalty=2.0,  # 높은 존재 페널티 설정
)

# 낮은 존재 페널티 설정에서의 응답 생성
response_low_presence_penalty = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=200,
    presence_penalty=0,  # 낮은 존재 페널티 설정
)

# 각 응답 내용 저장
content_high_frequency_penalty = \
    response_high_frequency_penalty.choices[0].message.content
content_low_frequency_penalty = \
    response_low_frequency_penalty.choices[0].message.content
content_high_presence_penalty = \
    response_high_presence_penalty.choices[0].message.content
content_low_presence_penalty = \
    response_low_presence_penalty.choices[0].message.content

# 결과 출력 (콘솔에 출력)
print("높은 빈도 페널티:")
print(prefix + content_high_frequency_penalty)
print()
print("낮은 빈도 페널티:")
print(prefix + content_low_frequency_penalty)
print()
print("높은 존재 페널티:")
print(prefix + content_high_presence_penalty)
print()
print("낮은 존재 페널티:")
print(prefix + content_low_presence_penalty)


높은 빈도 페널티:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸른 산과 맑은 강으로 둘러싸여 있었고, 사람들은 평화롭게 살았습니다. 하지만 이 마을에는 특별한 전설이 하나 있었지요.

전설에 따르면, 매년 가을이면 저 멀리 있는 신비로운 숲 속에서 '빛나는 단풍 열매'가 열린다고 했습니다. 이 열매를

낮은 빈도 페널티:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸르고 울창한 숲과 흐르는 맑은 강, 그리고 사계절 내내 꽃이 만개하는 아름다운 곳이었습니다. 하지만 그 마을에는 하나의 비밀이 숨어 있었습니다.

마을 한가운데에는 오래된 고목나무가 있었는데, 이 나무는 마을 사람들에게 특별한 의미를 지니고 있

높은 존재 페널티:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 아름다운 숲과 흐르는 강 사이에 자리 잡고 있었고, 주민들은 서로를 잘 알고 지냈습니다. 그러나 이 마을에는 하나의 비밀이 숨겨져 있었습니다.

마을 근처에는 오래된 성이 있었는데, 전설에 따르면 그 성 안에는 누구도 보지 못한 신비로운 금빛 용이 살고 있다고 했습니다. 사람들은 그 용이 엄청난 힘을 가지고 있으며, 소원을 들어줄 수 있다는 이야기를 들었지만, 아무도 성에 가려고 하지 않았습니다. 왜냐하면 용에게 다가가는 것은 위험한 일이라고 믿었기 때문입니다.

그러던 어느 날, 용감한 소년인 리오는 친구들과 함께 그 용을 보기로 결심했습니다. "우리가 갑자기 나타나면 무서울 거야!"라고 말하며 친구들을

낮은 존재 페널티:
옛날 옛적에 옛날 옛적에, 한 작은 마을에 조용하고 평화로운 숲이 있었습니다. 이 숲은 신비로운 생물과 다채로운 꽃들로 가득 차 있었고, 마을 사람들은 그곳에서 많은 전설을 가지고 살았죠.

그 숲의 깊은 곳에는 '별빛 요정'이라 불리는 작은 요정이 살고 있었습니다. 그녀는 매일 밤 별빛을 모아 마을 사람들에게 꿈과 희망을 가져다주었고, 사람들은 그녀의 존재를 믿고 따랐습니다. 하지만 어느 날, 별빛 

### 6-6. n매개변수를 활용한 여러 응답 생성: 한 번에 여러 텍스트 결과 얻기

In [9]:
# 사용할 모델 설정
model = "gpt-4o-mini"

# 사용자 입력 설정
prefix = "옛날 옛적에 "

# 시스템과 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 이야기꾼입니다."  # 시스템 역할 설정: 이야기꾼
    },
    {
        "role": "user",
        "content": prefix  # 사용자 입력
    },
]

# 응답 생성 요청, 스트리밍 활성화
response = client.chat.completions.create(
   model=model,
   messages=messages,
   n = 2,
   stop=["\n"]
)

choices = response.choices
for choice in choices:
    print(f"Choice: {choice.index}")
    print(prefix + choice.message.content)
    print()

Choice: 0
옛날 옛적에 옛날 옛적에, 깊은 숲 속에 작은 마을이 있었습니다. 이 마을은 아름다운 꽃들과 맑은 시냇물이 흐르는 곳이었고, 주민들은 평화롭게 살고 있었습니다. 마을 한가운데에는 거대한 오래된 나무가 서 있었는데, 그 나무는 마을 사람들에게 신성한 존재로 여겨졌습니다. 

Choice: 1
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸르른 산과 맑은 강으로 둘러싸여 있었고, 사람들은 서로 돕고 사는 따뜻한 공동체였습니다. 마을의 중심에는 작은 나무로 지어진 집이 있었고, 그 집에는 항상 웃음과 기쁨이 넘쳤습니다.



## 7장 GPT 모델의 성능을 극대화하는 프롬프트 엔지니어링 기법

### 7-2. Few Shot Learning: 프롬프트 엔지니어링의 핵심 기법

In [10]:
model = "gpt-4o-mini"
messages = [
    {
        "role": "system",
        "content":
        "당신은 텍스트에서 키워드를 추출하는 데 도움을 주는 유용한 봇입니다. "
        "키워드는 텍스트에서 중요한 단어들입니다."
    },
    {
        "role": "user",
        "content":
        "한 기발한 마을에서, 늙은 거북이 오리온이 언덕 위에 해바라기 씨앗을 뿌렸습니다. "
        "그는 이 씨앗들이 별들에게 소원을 전달한다고 믿었습니다. "
        "그가 모르는 사이에, 아래에 있던 아이들은 기뻐했습니다. "
        "이 씨앗들이 하늘에서 내린 축복이라고 믿었기 때문입니다. "
        "때로는 단순한 행동이 마법을 만들어냅니다."
    },
    {
        "role": "assistant",
        "content":
        "[\"기발한 마을\", \"늙은 거북이\", "
        "\"오리온\", \"해바라기 씨앗\", "
        "\"언덕\", \"소원\", \"별들\", "
        "\"아이들\", \"축복\", \"하늘\", "
        "\"단순한 행동\", \"마법\"]"
    },
    {
        "role": "user",
        "content":
        "양자 컴퓨팅 세계에서 하드론 처리 장치(HPU)가 돋보입니다. "
        "큐비트 중첩을 활용하여 뛰어난 속도를 제공합니다. "
        "나노광자 회로와 결합하여 빠른 큐비트 통신을 보장하며, "
        "양자 터널링은 오류 수정에 도움을 줍니다. "
        "양자 어닐링 알고리즘을 통해 HPU는 우리를 양자 우위에 한 걸음 더 가깝게 만듭니다."
    },
    {
        "role": "assistant",
        "content":
        "[\"양자 컴퓨팅\", \"하드론 처리 장치\", "
        "\"HPU\", \"큐비트 중첩\", \"속도\", "
        "\"나노광자 회로\", \"큐비트 통신\", "
        "\"양자 터널링\", \"오류 수정\", "
        "\"양자 어닐링 알고리즘\", "
        "\"양자 우위\"] "
    },
    {
        "role": "user",
        "content":
        "최초로 발명된 프로그래밍 언어는 Plankalkül로, "
        "1940년대에 Konrad Zuse가 설계했지만 1972년까지 공개적으로 "
        "알려지지 않았습니다(그리고 1998년까지 구현되지 않았습니다). "
        "널리 알려지고 성공한 최초의 고급 프로그래밍 언어는 "
        "Fortran으로, 1954년부터 1957년까지 John Backus가 이끄는 "
        "IBM 연구원 팀에 의해 개발되었습니다. FORTRAN의 성공으로 "
        "보편적인 컴퓨터 언어를 개발하기 위한 과학자 위원회가 구성되었고, "
        "그 노력의 결과로 ALGOL 58이 탄생했습니다. "
        "별도로, MIT의 John McCarthy는 학계에서 기원하여 성공한 최초의 "
        "언어인 Lisp를 개발했습니다. 이러한 초기 노력들의 성공으로, "
        "프로그래밍 언어는 1960년대 이후 활발한 연구 주제가 되었습니다."
    },
]

response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    temperature=0,
)
print(response.choices[0].message.content)


["프로그래밍 언어", "Plankalkül", "Konrad Zuse", "1940년대", "1972년", "Fortran", "1954년", "1957년", "John Backus", "IBM", "과학자 위원회", "ALGOL 58", "MIT", "John McCarthy", "Lisp", "1960년대", "연구 주제"]


In [11]:
# app.py : 암호화폐 정보 생성
# 사용법: python3 app.py [암호화폐 이름]

import sys  # 시스템 명령어 사용을 위한 모듈

crypto = input("암호화폐 이름을 입력해 주세요 : ")

# 사용할 모델 설정
model = "gpt-4o-mini"

# 시스템 및 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 스마트 어시스턴트입니다. 아래 답변 형식에 맞추어 모르는 것은 검색해서 알려주렴."  # 시스템 역할 설정
    },
    {
        "role": "user",
        "content": "Bitcoin",  # 사용자 입력: 비트코인 정보 요청
    },
    {
        "role": "assistant",
        "content": (
            "- BTC는 2008년에 생성되었습니다.\n"
            "- 자세한 정보는 여기서 확인할 수 있습니다:\n"
            "https://bitcoin.org/en/\n"
            "- 최신 가격은 여기서 확인할 수 있습니다:\n"
            "https://www.coingecko.com/en/coins/bitcoin\n"
            "- 최고가는 $64,895.00입니다.\n"
            "- 최저가는 $67.81입니다.\n"
        )
    },
    {
        "role": "user",
        "content": "Ethereum",  # 사용자 입력: 이더리움 정보 요청
    },
    {
        "role": "assistant",
        "content": (
            "- ETH는 2015년에 생성되었습니다.\n"
            "- 자세한 정보는 여기서 확인할 수 있습니다:\n"
            "https://ethereum.org/en/\n"
            "- 최신 가격은 여기서 확인할 수 있습니다:\n"
            "https://www.coingecko.com/en/coins/ethereum\n"
            "- 최고가는 $4,362.35입니다.\n"
            "- 최저가는 $0.43입니다.\n"
        )
    },
    {
        "role": "user",
        "content": "Dogecoin",  # 사용자 입력: 도지코인 정보 요청
    },
    {
        "role": "assistant",
        "content": (
            "- DOGE는 2013년에 생성되었습니다.\n"
            "- 자세한 정보는 여기서 확인할 수 있습니다:\n"
            "https://dogecoin.com/\n"
            "- 최신 가격은 여기서 확인할 수 있습니다:\n"
            "https://www.coingecko.com/en/coins/dogecoin\n"
            "- 최고가는 $0.73입니다.\n"
            "- 최저가는 $0.00008690입니다.\n"
        )
    },
    {
        "role": "user",
        "content": crypto,  # 사용자 입력: 명령어 인자로 받은 암호화폐 정보 요청
    }
]

# 응답 생성 요청
response = client.chat.completions.create(
    model=model,
    messages=messages,
)

# 응답 내용 출력
output = response.choices[0].message.content.strip()
print(output)


암호화폐 이름을 입력해 주세요 : XRP
- XRP는 2012년에 생성되었습니다.
- 자세한 정보는 여기서 확인할 수 있습니다:
https://ripple.com/xrp/
- 최신 가격은 여기서 확인할 수 있습니다:
https://www.coingecko.com/en/coins/ripple
- 최고가는 $3.84입니다.
- 최저가는 $0.00268631입니다.


### 7-3. 프롬프트 체이닝으로 트윗 생성 최적화

In [12]:
model = "gpt-4o"
# 첫 번째 프롬프트: 해시태그를 추출하여 파이썬 리스트로 반환
messages = [
    {
        "role": "system",
        "content":
        "당신은 사람들이 트위터에서 사용할 텍스트에서 "
        "해시태그를 추출하도록 도와주는 유용한 봇입니다.",
    },
    {
        "role": "user",
        "content":
        "한 작은 마을에서, 늙은 거북 오리온이 언덕 위에 "
        "해바라기 씨앗을 뿌렸습니다. 그는 이 씨앗들이 "
        "별에 소원을 담고 있다고 믿었습니다. 하지만 "
        "아이들은 이 씨앗들이 하늘에서 온 축복이라 "
        "생각하며 기뻐했습니다. 때로는 작은 행동이 마법을 만듭니다.",
    },
    {
        "role": "assistant",
        "content": "#소원 #축복 #마법",
    },
    {
        "role": "user",
        "content":
        "양자 컴퓨팅 세계에서 하드론 처리 장치(HPU)는 "
        "두드러집니다. 큐비트 중첩을 활용하여 뛰어난 "
        "속도를 제공하며, 나노포토닉 회로와 결합하여 "
        "빠른 큐비트 통신을 보장합니다. 양자 터널링은 "
        "오류 수정을 돕습니다. 양자 어닐링 알고리즘을 통해 "
        "HPU는 양자 우월성에 한 걸음 더 가까워지고 있습니다.",
    },
    {
        "role": "assistant",
        "content": "#양자컴퓨팅 #큐비트 #양자알고리즘",
    },
    {
        "role": "user",
        "content":
        "좋은 친구는 당신이 필요할 때 항상 옆에 있어주는 "
        "사람입니다. 그들은 당신의 문제를 들어주고 "
        "해결책을 찾는 데 도움을 줍니다. 당신이 가장 "
        "힘들 때 옆에 있어주는 사람이 진정한 친구입니다.",
    },
    {
        "role": "assistant",
        "content": "#우정 #우정명언 #응원",
    },
    {
        "role": "user",
        "content":
        "최초로 발명된 프로그래밍 언어는 Plankalkül로, "
        "1940년대에 Konrad Zuse가 설계했지만 1972년까지 공개적으로 "
        "알려지지 않았습니다(그리고 1998년까지 구현되지 않았습니다). "
        "널리 알려지고 성공한 최초의 고급 프로그래밍 언어는 "
        "Fortran으로, 1954년부터 1957년까지 John Backus가 이끄는 "
        "IBM 연구원 팀에 의해 개발되었습니다. FORTRAN의 성공으로 "
        "보편적인 컴퓨터 언어를 개발하기 위한 과학자 위원회가 구성되었고, "
        "그 노력의 결과로 ALGOL 58이 탄생했습니다. "
        "별도로, MIT의 John McCarthy는 학계에서 기원하여 성공한 최초의 "
        "언어인 Lisp를 개발했습니다. 이러한 초기 노력들의 성공으로, "
        "프로그래밍 언어는 1960년대 이후 활발한 연구 주제가 되었습니다."
    },
]
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    temperature=0,
    stop=["\n", "assistant:", "user:"],
)
hashtags = response.choices[0].message.content

# 두 번째 프롬프트: 트윗 생성
# 해시태그를 입력으로 제공하여 모델이 트윗을 생성합니다.
# 예시는 모델이 트윗의 길이와 스타일을 유지할 수 있도록
# 가이드 역할을 합니다.
messages = [
    {
        "role": "system",
        "content":
        "당신은 사용자가 주어진 텍스트를 기반으로 "
        "해시태그를 포함한 트윗을 작성하는 "
        "지능적인 어시스턴트입니다. 트윗은 100자에서 "
        "280자 사이여야 합니다.",
    },
    {
        "role": "user",
        "content":
        "OpenAI는 대형 언어 모델의 한계를 넓히고 있으며, 이를 "
        "대중과 기업 모두가 쉽게 접근할 수 있도록 하고 있습니다. "
        "모델은 사람처럼 자연스러운 텍스트를 생성하고, 글쓰기를 "
        "돕고, 질문에 답하며 더 많은 일을 수행할 수 있습니다. "
        "#OpenAI #AI #언어모델",
    },
    {
        "role": "assistant",
        "content":
        "OpenAI는 대중과 기업을 위해 대형 언어 모델을 "
        "개선하고 있습니다. 모델은 자연스러운 텍스트를 "
        "생성하고, 글쓰기를 돕고, 질문에 답할 수 있습니다. "
        "#OpenAI #AI #언어모델",
    },
    {
        "role": "user",
        "content":
        "에펠탑은 프랑스 파리에 위치한 세계적으로 유명한 "
        "건축물이며 사랑의 상징입니다. 매년 많은 관광객들이 "
        "그 웅장함과 낭만적인 분위기를 체험하기 위해 방문합니다. "
        "#에펠탑 #파리 #낭만",
    },
    {
        "role": "assistant",
        "content":
        "파리의 에펠탑은 사랑의 상징으로, 전 세계 관광객들이 "
        "그 웅장함을 보기 위해 모입니다. "
        "#에펠탑 #파리 #사랑",
    },
    {
        "role": "user",
        "content":
        "요가는 인도에서 유래된 고대의 수련법으로, 정신적, "
        "육체적, 영적인 안녕을 목표로 합니다. 요가는 전신의 "
        "건강을 촉진하는 운동과 명상을 제공합니다. "
        "#요가 #웰빙 #명상",
    },
    {
        "role": "assistant",
        "content":
        "요가는 인도에서 유래된 수련법으로, 정신과 신체, 영혼을 "
        "균형 있게 조화시킵니다. "
        "#요가 #전신건강 #명상",
    },
    {
        "role": "user",
        "content":
        "팬더는 중국 남중부에서 서식하며, 독특한 흑백 외형으로 "
        "잘 알려져 있습니다. 팬더는 주로 대나무를 먹으며, 평화와 "
        "보존 노력을 상징하는 동물입니다. "
        "#팬더 #중국 #보존",
    },
    {
        "role": "assistant",
        "content":
        "팬더는 중국이 원산지이며, 평화와 보존을 상징하는 "
        "아이콘입니다. "
        "#팬더 #중국 #평화",
    },
    {
        "role": "user",
        "content":
        "그랜드 캐니언은 미국 애리조나 주에 위치한 콜로라도 강에 "
        "의해 형성된 가파른 협곡입니다. 이 협곡은 거대한 규모와 "
        "복잡하고 다채로운 풍경으로 유명한 명소입니다. "
        "#그랜드캐니언 #애리조나 #자연",
    },
    {
        "role": "assistant",
        "content":
        "애리조나의 콜로라도 강에 의해 형성된 그랜드 캐니언은 "
        "그 거대한 규모와 다채로운 풍경으로 유명합니다. "
        "#그랜드캐니언 #애리조나 #자연",
    },
    {
        "role": "user",
        "content":
        "최초로 발명된 프로그래밍 언어는 Plankalkül로, "
        "1940년대에 Konrad Zuse가 설계했지만 1972년까지 공개적으로 "
        "알려지지 않았습니다(그리고 1998년까지 구현되지 않았습니다). "
        "널리 알려지고 성공한 최초의 고급 프로그래밍 언어는 "
        "Fortran으로, 1954년부터 1957년까지 John Backus가 이끄는 "
        "IBM 연구원 팀에 의해 개발되었습니다. FORTRAN의 성공으로 "
        "보편적인 컴퓨터 언어를 개발하기 위한 과학자 위원회가 구성되었고, "
        "그 노력의 결과로 ALGOL 58이 탄생했습니다. "
        "별도로, MIT의 John McCarthy는 학계에서 기원하여 성공한 최초의 "
        "언어인 Lisp를 개발했습니다. 이러한 초기 노력들의 성공으로, "
        "프로그래밍 언어는 1960년대 이후 활발한 연구 주제가 "
        "되었습니다. {hashtags}",
    },
]

# 280자 이하의 트윗을 5개 생성하고 첫 번째 트윗을 선택
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=280,
    temperature=0.5,
    stop=["\n", "assistant:", "user:"],
    n=5
)
for choice in response.choices:
    tweet = choice.message.content
    length = len(tweet)
    if length <= 280:
        print(tweet)
        print()
        break


프로그래밍 언어의 역사는 Plankalkül에서 시작되었으며, Fortran과 Lisp의 성공으로 활발한 연구가 이어졌습니다. #프로그래밍언어 #Fortran #Lisp #컴퓨터과학 #기술역사



### 7-4. GPT로 올드스쿨 랩 가사 완성하기: General Knowledge Prompting 활용법

In [13]:
model = "gpt-4o-mini"
# 첫 번째 프롬프트로 지식 생성
prompt = """
올드스쿨 랩의 가사적 특징과
주제에 대해 간결한 문단을 작성하세요.
"""
messages = [
    {
        "role": "system",
        "content": "당신은 스마트 어시스턴트입니다.",  # 시스템 역할 설정
    },
    {
        "role": "user",
        "content": prompt,  # 사용자 입력: 올드스쿨 랩에 대한 질문
    }
]
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=200,  # 생성할 최대 토큰 수
    temperature=0.5,  # 창의성을 제한하는 온도 설정
    stop=["assistant:", "user:"],  # 멈춤 조건
)
output = response.choices[0].message.content

# 두 번째 프롬프트로 가사 생성
# 첫 번째 프롬프트의 출력을 두 번째 프롬프트의
# 입력으로 사용하여 더 나은 응답을 생성
prompt = f"""배경: {output}

작업: 정의와 평등에 대한
올드스쿨 랩 노래 가사를 작성하세요.
"""
messages = [
    {
        "role": "system",
        "content":
        "당신은 유명한 올드스쿨 랩 작사가입니다.",  # 시스템 역할 설정: 랩 작사가
    },
    {
        "role": "user",
        "content": prompt,  # 사용자 입력: 정의와 평등에 대한 랩 가사 생성 요청
    },
]
# 5개의 트윗을 생성하고 280자 이하인 첫 번째 트윗을 선택
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=500,  # 생성할 최대 토큰 수
    temperature=1,  # 높은 창의성 설정
    stop=["assistant:", "user:"],  # 멈춤 조건 설정
)
output = response.choices[0].message.content
print(
    "모델에게 전달한 프롬프트는 다음과 같습니다"
    f":\n\n{prompt}"
)
print()
print(f"다음은 결과입니다:\n\n{output}")


모델에게 전달한 프롬프트는 다음과 같습니다:

배경: 올드스쿨 랩은 1970년대 후반부터 1980년대 초반에 걸쳐 발전한 힙합의 초기 형태로, 주로 간결하고 리드미컬한 비트 위에 래퍼의 독창적인 플로우가 특징입니다. 가사적 특징으로는 일상적인 경험, 사회적 불평등, 그리고 도시 생활의 현실을 다룬 내용이 많습니다. 또한, 유머와 자아 표현이 두드러지며, 종종 브레이크 댄스와 같은 문화적 요소와 연결됩니다. 주제적으로는 정체성, 공동체 의식, 그리고 사회적 메시지를 전달하는 데 중점을 두며, 이는 당시의 사회적 맥락과 긴밀하게 연관되어 있습니다. 이러한 요소들은 올드스쿨 랩이 힙합 문화의 기초를 형성하는 데 중요한 역할을 했음을 보여

작업: 정의와 평등에 대한
올드스쿨 랩 노래 가사를 작성하세요.


다음은 결과입니다:

**제목: 정의와 평등의 거리**

(Verse 1)  
이 도시의 거리는 요란한 소리  
우린 매일매일 힘겹게 길을 걷지,  
어둠 속에 숨은 불빛, 여전히 돋보여  
정의는 먼 곳, 하지만 우린 항상 소리쳐

(Chorus)  
정의와 평등, 이름을 외쳐봐  
우리의 목소리로 세상을 바꿔가  
어깨를 나란히, 하나로 일어나  
이건 우리의 싸움, 함께 할 때 더 강해져

(Verse 2)  
부자와 가난한 사람의 경계선은 낯설어  
왜 한쪽에만 기회가 넘어가냐고 사람들아  
학교의 종소리는 평등이 아닌 불공정  
우리에게는 꿈이 있어, 하지만 현실은 거칠어

(Chorus)  
정의와 평등, 이름을 외쳐봐  
우리의 목소리로 세상을 바꿔가  
어깨를 나란히, 하나로 일어나  
이건 우리의 싸움, 함께 할 때 더 강해져

(Bridge)  
브레이크 댄스처럼, 고난에 춤을 춰  
소외된 합창, 우린 그 해답을 찾아  
두 손을 모아, 함께 세운 이 바닥  
우리의 꿈을 위해 계속해서 나아가

(Verse 3)  
차별과 고통, 더는 못 참겠어  
우리가 세운 이 길, 함께 걸을 거야  
푸른 하늘 아래, 그늘이 지워지길  
정의는 가깝다, 

### 7-5. 프롬프트의 힘: 문맥 채우기로 AI의 이해도 향상시키기

In [14]:
model = "gpt-4o-mini"  # 사용할 모델 지정
prompt = "단어 'light'의 품사를 결정하세요."  # 프롬프트를 한글로 변경

messages = [
    {
        "role": "system",
        "content": "당신은 똑똑한 어시스턴트입니다.",  # 시스템 메시지를 한글로 변경
    },
    {
        "role": "user",
        "content": prompt,
    }
]

# API를 통해 채팅 완성 요청
response = client.chat.completions.create(
    model=model,
    messages=messages,
)

# 응답에서 출력 내용 추출
output = response.choices[0].message.content

# 결과 출력
print(output)


단어 'light'는 여러 품사로 사용될 수 있습니다. 주요 품사는 다음과 같습니다:

1. **명사 (Noun)**: 빛, 조명.
   - 예: "The light in the room is very bright."

2. **형용사 (Adjective)**: 가벼운, 밝은.
   - 예: "She wore a light jacket."

3. **동사 (Verb)**: 불을 붙이다, 비추다.
   - 예: "Please light the candle."

따라서 'light'의 품사는 문맥에 따라 달라질 수 있습니다.


In [15]:
# 사용할 모델 지정
model = "gpt-4o-mini"

# 각 상황에 대한 프롬프트 정의
prompt_a = """The light is red. Determine the part
of speech of the word 'light'.\n\n"""

prompt_b = """This desk is very light. Determine
the part of speech of the word 'light'.\n\n"""

prompt_c = """You light up my life. Determine the
part of speech of the word 'light'.\n\n"""

prompt_d = """He stepped light on the snow, trying
not to leave deep footprints. Determine the part of
speech of the word 'light'.\n\n"""

# 각 프롬프트에 대해 반복
for prompt in [prompt_a, prompt_b, prompt_c, prompt_d]:
    # 메시지 구성
    messages = [
        {
            "role": "system",
            "content": "You are a smart assistant.",
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]

    # API를 통해 응답 요청
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    # 응답 출력
    output = response.choices[0].message.content
    print(output)
    print()  # 각 응답 사이에 빈 줄 추가

In the phrase "The light is red," the word "light" is a noun. It refers to the physical object or source of illumination.

In the sentence "This desk is very light," the word "light" functions as an adjective. It describes the noun "desk," indicating that the desk has a low weight.

In the phrase "You light up my life," the word "light" functions as a verb. It indicates the action of illuminating or bringing light to something.

In the sentence "He stepped light on the snow, trying not to leave deep footprints," the word "light" functions as an adjective. It describes the manner in which he stepped, indicating that he is stepping gently or not heavily.



In [16]:
# 사용할 모델 지정
model = "gpt-4o-mini"

# 프롬프트 정의 (첫 번째는 Apple이 회사, 두 번째는 Apple이 회사 또는 과일로 인식되도록)
prompt_a = """
Huawei:
company

Google:
company

Microsoft:
company
Apple:
"""
prompt_b = """
Huawei:
company

Google:
company

Microsoft:
company

Apricot:
Fruit

Apple:
"""

# 각 프롬프트에 대해 반복 처리
for prompt in [prompt_a, prompt_b]:
    # 메시지 구성
    messages = [
        {
            "role": "system",
            "content": "당신은 스마트한 어시스턴트입니다.",
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]

    # API 호출을 통해 응답 생성
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    # 응답 출력
    output = response.choices[0].message.content
    print(output)  # 출력 후 빈 줄 추가
    print()


Apple: company

Fruit



In [17]:
# 사용할 모델 지정
model = "gpt-4o-mini"

# Python 애플리케이션을 위한 Dockerfile 생성 프롬프트
prompt_dockerfile = """
# Node.js용 Dockerfile:
FROM node:14
WORKDIR /app
COPY . /app
RUN npm install
EXPOSE 8080
CMD ["node", "app.js"]
# Python용 Dockerfile:
"""

# MySQL 데이터베이스를 위한 Kubernetes 배포 스크립트 생성 프롬프트
prompt_kubernetes = """
# Redis용 Kubernetes 배포:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: redis-deployment
spec:
  selector:
    matchLabels:
      app: redis
  template:
    metadata:
      labels:
        app: redis
    spec:
      containers:
      - name: redis
        image: redis
        ports:
        - containerPort: 6379
# MySQL용 Kubernetes 배포:
"""

# 두 프롬프트에 대해 반복 처리
for prompt in [prompt_dockerfile, prompt_kubernetes]:
    # 메시지 구성
    messages = [
        {
            "role": "system",
            "content": "당신은 구성 스크립트를 작성하는 스마트한 어시스턴트입니다.",
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]

    # API 호출을 통해 응답 생성
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    # 응답 출력 및 불필요한 공백 제거 후 출력
    output = response.choices[0].message.content
    print(output.strip())
    print("---")  # 구분선 출력


다음은 Python 애플리케이션을 위한 Dockerfile 샘플입니다. 이 예시는 Flask를 사용하는 기본 웹 애플리케이션을 가정하고 작성되었습니다.

```dockerfile
# Python용 Dockerfile
FROM python:3.9

# 작업 디렉토리 설정
WORKDIR /app

# requirements.txt 파일을 복사하여 필요한 패키지를 설치합니다.
COPY requirements.txt /app/
RUN pip install --no-cache-dir -r requirements.txt

# 애플리케이션 코드를 복사합니다.
COPY . /app

# Flask 서버가 실행될 포트를 노출합니다.
EXPOSE 5000

# 애플리케이션을 시작하는 명령어를 설정합니다.
CMD ["flask", "run", "--host=0.0.0.0"]
```

이 Dockerfile을 사용할 때, `requirements.txt` 파일에 필요한 Python 패키지(예: Flask 등)를 명시해주셔야 합니다. 이 파일은 Docker build 명령어를 실행할 때 필요합니다. `app.py` 파일이 애플리케이션의 진입점이면, dockerfile의 `CMD` 부분에서 `app.py`를 실행하도록 변경할 수 있습니다.

써보고 싶은 사항이나 추가적인 요구 사항이 있으면 말씀해 주세요!
---
MySQL용 Kubernetes 배포를 추가하기 위한 구성 스크립트는 다음과 같습니다. 이 스크립트는 MySQL의 기본 배포 및 서비스 설정을 포함합니다.

```yaml
# MySQL용 Kubernetes 배포:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: mysql-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      app: mysql
  template:
    metadata:
      labels:
        app: mysql
    spec

### 7-6. 동적 토큰 제어: AI 응답 길이를 유연하게 조절하는 프롬프트 기법

In [18]:
import sys
model = "gpt-4o"

# 명령줄에서 할 일 항목의 개수를 읽어오기
print("""
Please enter the number of tasks.
Example :
number_of_tasks: 5
number_of_tasks: 10
""")

number_of_tasks = int(input("number_of_tasks: "))

# 모델이 지정된 형식으로 할 일 목록을 만들도록 안내하는 프롬프트 생성
prompt = """
Please create a todo list for establishing a company in the United States.
Each task should be written in one line.
Task 1: [task 1]
Task 2: [task 2]
Task 3: [task 3]
...
Task n: [task n]
"""


# 종료 시퀀스 정의
# 사용자가 5개의 할 일을 생성하고자 할 때, 종료 시퀀스는 "Task 6:"을 포함해야 함
stop = [
    f"Task {number_of_tasks + 1}:",
    "assistant:",
    "user:"
]

# 메시지 구성
messages = [
    {
        "role": "system",
        "content": "You are a smart assistant.",
    },
    {
        "role": "user",
        "content": prompt,
    }
]

# 모델에 요청하여 응답 생성
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=400,
    stop=stop,
)

# "Task 1:"과 생성된 출력물 연결
output = response.choices[0].message.content
print(output)



Please enter the number of tasks.
Example :
number_of_tasks: 5
number_of_tasks: 10

number_of_tasks: 6
Task 1: Conduct market research to validate business idea and demand.  
Task 2: Create a comprehensive business plan outlining mission, vision, and goals.  
Task 3: Choose a business name and verify its availability.  
Task 4: Decide on the legal structure (e.g., LLC, Corporation, Partnership).  
Task 5: Register your business with the state government.  
Task 6: Obtain all necessary business permits and licenses.  



In [19]:
import sys
model = "gpt-4o"

# 명령줄에서 할 일 항목의 개수를 읽어오기
print("""
Please enter the number of tasks.
Example :
number_of_tasks: 5
number_of_tasks: 10
""")

number_of_tasks = int(input("number_of_tasks: "))

# 모델이 지정된 형식으로 할 일 목록을 만들도록 안내하는 프롬프트 생성
prompt = """
Please create a todo list for establishing a company in the United States.
Each task should be written in one line.
Task 1: [task 1]
Task 2: [task 2]
Task 3: [task 3]
...
Task n: [task n]
"""

# 종료 시퀀스 정의
# 사용자가 5개의 할 일을 생성하고자 할 때, 종료 시퀀스는 "Task 6:"을 포함해야 함
stop = [
    f"Task {number_of_tasks + 1}:",
    "assistant:",
    "user:"
]

# 최대 토큰 수를 정의
max_tokens = number_of_tasks * 53 + 30  # 작업 수에 따라 최대 토큰 수 계산

# 메시지 구성
messages = [
    {
        "role": "system",
        "content": "You are a smart assistant.",
    },
    {
        "role": "user",
        "content": prompt,
    }
]

# 모델에 요청하여 응답 생성
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=max_tokens,
    stop=stop,
)

# 결과 출력
output = response.choices[0].message.content
print(output)


Please enter the number of tasks.
Example :
number_of_tasks: 5
number_of_tasks: 10

number_of_tasks: 6
Task 1: Conduct market research to identify business opportunities and competitors.  
Task 2: Develop a comprehensive business plan outlining goals, strategies, and financial projections.  
Task 3: Choose a business structure (e.g., LLC, corporation, partnership, sole proprietorship).  
Task 4: Register the business with the appropriate state authorities and obtain an Employer Identification Number (EIN) from the IRS.  
Task 5: Register your business name, if necessary, and secure a domain name for your business website.  
Task 6: Open a business bank account to manage financial transactions separately from personal finances.  



### 7-7. 프롬프트 템플릿 최적화: 리눅스 학습을 위한 AI 기반 CLI 어시스턴트 개발

In [20]:
# 사용할 모델 지정
model = "gpt-4o"

# 메시지 구성
messages = [
    {
        "role": "system",
        "content": "당신은 스마트한 어시스턴트입니다. 답변은 명령줄의 내용만 해 주세요."
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -l"
    },
    {
        "role": "user",
        "content": "숨김 파일을 포함하여 현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -la"
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 삭제해 주세요."
    },
    {
        "role": "assistant",
        "content": "rm *"
    },
    {
        "role": "user",
        "content": "파일 'test.txt'에서 'sun'이라는 단어가 몇 번 등장하는지 세어 주세요."
    },
    {
        "role": "assistant",
        "content": "grep -o 'sun' test.txt | wc -l"
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 파일 수를 세어주세요."
    }
]

# API 호출을 통해 응답 생성
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=200,
    temperature=0,  # 출력의 일관성을 위해 온도 값을 0으로 설정
)

# 응답에서 내용 추출 및 출력
output = response.choices[0].message.content.strip()
print(output)


ls -1 | wc -l


In [1]:
!pip install click==8.1.3

In [ ]:
import click
model = "gpt-4o"

# 기본 메시지 목록 정의
base_messages = [
    {
        "role": "system",
        "content": "당신은 스마트한 어시스턴트입니다. 답변은 명령줄의 내용만 해 주세요."
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -l"
    },
    {
        "role": "user",
        "content": "숨김 파일을 포함하여 현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -la"
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 삭제해 주세요."
    },
    {
        "role": "assistant",
        "content": "rm *"
    },
    {
        "role": "user",
        "content": "파일 'test.txt'에서 'sun'이라는 단어가 몇 번 등장하는지 세어 주세요."
    },
    {
        "role": "assistant",
        "content": "grep -o 'sun' test.txt | wc -l"
    },
]

# 무한 루프를 돌며 사용자 입력 처리
while True:
    # 메시지 목록 복사
    messages = base_messages.copy()

    # 사용자 입력을 읽음
    request = input(
        click.style(  # 'Input:' 프롬프트를 녹색으로 출력
            "Input: ",
            fg="green"
        )
    )

    # 사용자 입력을 메시지 목록에 추가
    messages.append(
        {
            "role": "user",
            "content": f"{request}"
        }
    )

    # API로 메시지를 전송
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=200,
        temperature=0,  # 결정론적 응답을 위한 온도 설정
    )

    # 응답에서 명령어 추출
    command = response.choices[0].message.content.strip()

    # 명령어를 보기 좋게 출력
    click.echo(
        click.style(
            "Output: ", fg="yellow"  # 'Output:' 프롬프트를 노란색으로 출력
        ) + command
    )
    click.echo()  # 빈 줄 추가

Input: 파일 삭제
Output: rm filename

Input: quit
Output: exit



KeyboardInterrupt: Interrupted by user

In [ ]:
import click
import os

model = "gpt-4o"

# 기본 메시지 목록 정의
base_messages = [
    {
        "role": "system",
       "content": "당신은 스마트한 어시스턴트입니다. 답변은 명령줄의 내용만 해 주세요."
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -l"
    },
    {
        "role": "user",
        "content": "숨김 파일을 포함하여 현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -la"
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 삭제해 주세요."
    },
    {
        "role": "assistant",
        "content": "rm *"
    },
    {
        "role": "user",
        "content": "파일 'test.txt'에서 'sun'이라는 단어가 몇 번 등장하는지 세어 주세요."
    },
    {
        "role": "assistant",
        "content": "grep -o 'sun' test.txt | wc -l"
    },
]

# 무한 루프 시작
while True:
    messages = base_messages.copy()  # 기본 메시지 복사

    # 사용자 입력 받기
    request = input(
        click.style(
            "Input (종료하려면 'exit' 입력): ",  # 입력 요청 메시지
            fg="green"
        )
    )

    # 사용자가 'exit' 또는 'quit'을 입력하면 프로그램 종료
    if request.lower() in ["exit", "quit"]:
        break

    # 사용자 입력을 메시지 목록에 추가
    messages.append(
        {
            "role": "user",
            "content": f"{request}"
        }
    )

    # API로 메시지 전송
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=200,
        temperature=0,  # 결정론적 응답을 위한 온도 설정
    )

    # 응답에서 명령어 추출
    command = response.choices[0].message.content.strip()

    # 명령어를 보기 좋게 출력
    click.echo(
        click.style(
            "Output: ", fg="yellow"
        ) + command
    )

    # 명령어를 실행할 것인지 묻기
    click.echo(
        click.style(
            "명령어를 실행하시겠습니까? (y/n): ",
            fg="yellow"
        ),
        nl=False
    )

    # 사용자 선택 입력
    choice = input()

    # 사용자가 'y'를 입력하면 명령어 실행
    if choice == "y":
        os.system(command)
    elif choice == "n":
        continue
    else:
        click.echo(
            click.style(
                "잘못된 선택입니다. 'y' 또는 'n'을 입력해 주세요.",
                fg="red"
            )
        )

    click.echo()  # 빈 줄 추가


Input (종료하려면 'exit' 입력): 파일 검색
Output: find . -name "filename"
명령어를 실행하시겠습니까? (y/n): y

Input (종료하려면 'exit' 입력): 빈 파일 하나 생성하기
Output: touch filename.txt
명령어를 실행하시겠습니까? (y/n): y

Input (종료하려면 'exit' 입력): filename.txt 삭제하기
Output: rm filename.txt
명령어를 실행하시겠습니까? (y/n): y

Input (종료하려면 'exit' 입력): exit
